In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd /content/gdrive/MyDrive/36114_ADSI/AT2

/content/gdrive/MyDrive/36114_ADSI/AT2


In [3]:
!pip install category_encoders

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 7.3 MB/s eta 0:00:00


In [4]:
import pandas as pd
import numpy as np
import category_encoders as ce
import tensorflow as tf
import matplotlib.pyplot as plt
from joblib import dump
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

In [ ]:
!pwd

/content/gdrive/MyDrive/36114_ADSI/AT2


In [5]:
# Read in the csv
df = pd.read_csv("/content/gdrive/MyDrive/36114_ADSI/AT2/beer_reviews.csv")


In [6]:
# Select necessary columns
df = df[['review_appearance', 'review_aroma', 'review_palate', 'review_taste', 'beer_abv', 'brewery_name', 'beer_style', 'beer_name']]
df.head()

,review_appearance,review_aroma,review_palate,review_taste,beer_abv,brewery_name,beer_style,beer_name
0,2.5,2.0,1.5,1.5,5.0,Vecchio Birraio,Hefeweizen,Sausa Weizen
1,3.0,2.5,3.0,3.0,6.2,Vecchio Birraio,English Strong Ale,Red Moon
2,3.0,2.5,3.0,3.0,6.5,Vecchio Birraio,Foreign / Export Stout,Black Horse Black Beer
3,3.5,3.0,2.5,3.0,5.0,Vecchio Birraio,German Pilsener,Sausa Pils
4,4.0,4.5,4.0,4.5,7.7,Caldera Brewing Company,American Double / Imperial IPA,Cauldron DIPA


In [7]:
# Handle missing values
df['beer_abv'].fillna(df['beer_abv'].median(), inplace=True)
df['brewery_name'].fillna('unknown', inplace=True)

In [8]:
# Encode the beer_style column
le = LabelEncoder()
df['beer_style'] = le.fit_transform(df['beer_style'])

In [9]:
# Select columns to be encoded
cols_to_encode = ['brewery_name','beer_name']

# Instantiate encoder
encoder = ce.TargetEncoder(cols=cols_to_encode)

In [10]:
# Fit and transform the columns
df_encoded = encoder.fit_transform(df[cols_to_encode], df['beer_style'])

# Replace original columns with encoded ones in the dataframe
df.drop(cols_to_encode, axis=1, inplace=True)
df = pd.concat([df, df_encoded], axis=1)

In [11]:
# Assuming that df_X contains your features and df_y contains your target
df_X = df.drop('beer_style', axis=1)
df_y = df['beer_style']

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df_X, df_y, test_size=0.2, random_state=42)

In [12]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten

In [14]:
# Initialize the constructor
model = Sequential()

# Input Layer
model.add(Dense(128, activation='relu', input_shape=(X_train.shape[1],)))

# Hidden Layers
model.add(Dense(100, activation='relu'))
model.add(Dropout(0.3)) # Dropout layer after each Dense layer
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.3))

# Output Layer
model.add(Dense(104, activation='softmax'))

# Compile the model
model.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [38]:
# model.compile(loss='sparse_categorical_crossentropy',
#               optimizer='SGD',
#               metrics=['accuracy'])


In [15]:
# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=256, validation_split=0.2, verbose=1)

Epoch 1/100
3967/3967 [==============================] - 19s 3ms/step - loss: 2.3266 - accuracy: 0.3685 - val_loss: 1.6683 - val_accuracy: 0.5243
Epoch 2/100
3967/3967 [==============================] - 13s 3ms/step - loss: 1.7882 - accuracy: 0.4960 - val_loss: 1.4694 - val_accuracy: 0.5949
Epoch 3/100
3967/3967 [==============================] - 13s 3ms/step - loss: 1.6766 - accuracy: 0.5294 - val_loss: 1.3907 - val_accuracy: 0.6311
Epoch 4/100
3967/3967 [==============================] - 13s 3ms/step - loss: 1.6172 - accuracy: 0.5490 - val_loss: 1.3803 - val_accuracy: 0.6159
Epoch 5/100
3967/3967 [==============================] - 13s 3ms/step - loss: 1.5776 - accuracy: 0.5624 - val_loss: 1.2927 - val_accuracy: 0.6560
Epoch 6/100
3967/3967 [==============================] - 13s 3ms/step - loss: 1.5474 - accuracy: 0.5729 - val_loss: 1.3511 - val_accuracy: 0.6343
Epoch 7/100
3967/3967 [==============================] - 13s 3ms/step - loss: 1.5242 - accuracy: 0.5819 - val_loss: 1.3743 -